## **Configurando TPU**

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

## **Clonando e instalando o Tensorflow API Object Detection** 

Para usar a API do Tensorflow para detecção de objetos é necessário clonar seu repositório no Github.


In [ ]:
!git clone https://github.com/tensorflow/models.git

A API do Tensorflow depende dos chamados `protocol buffers`, também conhecidos como `protobufs`.<br/>
Abaixo estamos compilando todos os `protobufs` na pasta `object_detection/protos` em Python.

In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

Instalando a API do Tensorflow

In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Executando o teste de construtor de modelo

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## **Conectando ao Google Drive**

In [ ]:
%cd /content/
from google.colab import drive
drive.mount('/content/gdrive')

Cria um link simbólico para facilitar acesso

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /content/mydrive
!ls /content/mydrive

Copia o dataset para a cloud VM

In [ ]:
!cp "/content/mydrive/{your path here}/dataset.zip /content
!unzip dataset.zip

##**Configurando Dataset**

Converte os rotulos de xml para csv

In [ ]:
%cd /content/dataset/
!python xml_to_csv.py

Gera os arquivos no formato tensorflow<br>
train.record e test.record

In [ ]:
%cd /content/dataset/
!python3 generate_tfrecord.py --csv_input=data/train_labels.csv --output_path=data/train.record --image_dir=train/
!python3 generate_tfrecord.py --csv_input=data/test_labels.csv --output_path=data/test.record --image_dir=test/

## **Mobilenet V2**

Baixando e descompactando a mobilenet v2 que utiliza o extrator de recursos SSD(single shot detection), disponível no repositorio do Tensorflow.

Junto ao modelo vem os pesos gerados do treinamento sobre o dataset COCO

In [ ]:
%cd /content
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!rm mobilenet_v2.tar.gz

Em seguida é necessário baixar o arquivo de configuração para o modelo. Este arquivo já vem configurado para utilizar em TPU.

É interessante renomear ele para facilitar

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

## **Configurações Google Cloud Service Bucket**

Setando variáveis para facilitar o acesso

In [ ]:
PROJECT = "Your Project Here"
YOUR_GCS_BUCKET = "Your GCS Bucket Here"

In [ ]:
!gcloud config set project '{PROJECT}'

Em seguida, criaremos o acesso ao bucket que queremos acessar

In [ ]:
!gsutil mb gs://'{YOUR_GCS_BUCKET}'

Autenticação ao Bucket

In [ ]:
!gcloud auth login

In [ ]:
!gcloud auth application-default login

Em seguida, daremos acesso a TPU da VM ao nosso repositório no Bucket adicionando a conta de acesso especifica para TPU.<br/> 
O comando abaixo retorna a conta de acesso de TPU.

In [ ]:
!curl -H "Authorization: Bearer $(gcloud auth print-access-token)" https://ml.googleapis.com/v1/projects/'{PROJECT}':getConfig

Define uma varivel com a conta de acesso de TPU

In [ ]:
TPU_ACCOUNT = "Your TPU Account Here"

In [ ]:
!gcloud projects add-iam-policy-binding '{PROJECT}' --member serviceAccount:'{TPU_ACCOUNT}' --role roles/ml.serviceAgent

## **Transferindo os dados para o GCS Bucket**

Primeiro mandamos os arquivos record, que nada mais são que o dataset convertido para o formato do tensorflow

In [ ]:
!gsutil -m cp -r /content/dataset/data/*.record gs://'{YOUR_GCS_BUCKET}'/data/

Envia o arquivo que contém as classes dos objetos para detecção

In [ ]:
!gsutil cp /content/dataset/data/object-detection.pbtxt gs://'{YOUR_GCS_BUCKET}'/data/object-detection.pbtxt

Checkpoints do modelo pré treinado

In [ ]:
!gsutil cp /content/mobilenet_v2/mobilenet_v2.ckpt* gs://'{YOUR_GCS_BUCKET}'/data/

Arquivo de configuração do modelo

In [ ]:
!gsutil cp /content/mobilenet_v2.config gs://'{YOUR_GCS_BUCKET}'/data/mobilenet_v2.config

## **Definindo os parâmetros de treinamento**

Copia o arquivo de configuração do bucket

In [ ]:
!gsutil cp gs://'{YOUR_GCS_BUCKET}'/data/mobilenet_v2.config /content/mobilenet_v2.config

Variaveis a serem editadas no arquivo

In [ ]:
num_classes = 2
batch_size = 256 
num_steps = 5000 
num_eval_steps = 1000

# Após primeiro treinamento, é necessário alterar para detection, 
# pois o treinamento é salvo como detection
fine_tune_checkpoint_type = "classification"

train_record_path = "gs://"+YOUR_GCS_BUCKET+"/data/train.record"
test_record_path = "gs://"+YOUR_GCS_BUCKET+"/data/test.record"

# Onde será salvo os checkpoints de treinamento
model_dir = "gs://"+YOUR_GCS_BUCKET+"/train" 
labelmap_path = "gs://"+YOUR_GCS_BUCKET+"/data/object-detection.pbtxt"

pipeline_config_path = "gs://"+YOUR_GCS_BUCKET+"/data/mobilenet_v2.config"

# Após primeiro treinamento altere para apontar para o ultimo checkpoint gerado
# Será algo parecido com isso "gs://"+YOUR_GCS_BUCKET+"/train/ckpt-n" sendo n o numero do ultimo checkpoint
fine_tune_checkpoint = "gs://"+YOUR_GCS_BUCKET+"/data/mobilenet_v2.ckpt-1"

Editando o arquivo de configuração


In [ ]:
import re

with open('/content/mobilenet_v2.config') as f:
    config = f.read()

with open('/content/mobilenet_v2.config', 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
                  'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint),config)
  
  # Set type fine tune
  config = re.sub('fine_tune_checkpoint_type: ".*?"',
                  'fine_tune_checkpoint_type: "{}"'.format(fine_tune_checkpoint_type),config)


  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  f.write(config)

Exibe o arquivo para verificar se as alterações foram realizadas

In [ ]:
%cat /content/mobilenet_v2.config

Envia o arquivo de configuração editado para o Bucket

In [ ]:
!gsutil cp /content/mobilenet_v2.config gs://'{YOUR_GCS_BUCKET}'/data/mobilenet_v2.config

## **Treinando o modelo**

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_config_path} \
  --model_dir={model_dir} \
  --use_tpu=True

## **Validação do modelo**

Assim que a avaliação terminar, será exibida a seguinte mensagem:<br/>
`INFO:tensorflow:Waiting for new checkpoint at /content/training/`<br/>
Então você já pode parar a execução da célula

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}

## **Métricas de treinamento e validação**

In [ ]:
%load_ext tensorboard
%tensorboard --logdir gs://'{YOUR_GCS_BUCKET}'/train